# Amazon SageMaker で PyTorch のコードを動かすサンプルノートブック
このノートブックは、PyTorch のソースコードを Amazon SageMaker で動かすサンプルです。

## このノートブックの使用上の注意
- このノートブックでは，学習にp2.xlargeを使用するためやや料金がかかります。インスタンスタイプごとの料金に関しては[こちら](https://aws.amazon.com/jp/sagemaker/pricing/)をご参照ください。
- このノートブックでは，約780MBの画像データをS3に保存しますので，不要になったら削除してください。

## 下準備
### Amazon SageMaker を使うための設定
セッション情報、画像を保存するバケット名、ロールの取得を行います。バケット名のデフォルトは、sagemaker-[リージョン]-[アカウントID] です。

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-srgan'

role = sagemaker.get_execution_role()


### 使用する画像のダウンロード
このノートブックでは，[COCOデータセット](http://cocodataset.org/#download)を使用します。

ダウンロードしたデータをtrainとvalidに分けて学習で使用します。

The annotations in this dataset belong to the COCO Consortium and are licensed under a Creative Commons Attribution 4.0 License. The COCO Consortium does not own the copyright of the images. Use of the images must abide by the Flickr Terms of Use. The users of the images accept full responsibility for the use of the dataset, including but not limited to the use of any copies of copyrighted images that they may create from the dataset. Before you use this data for any other purpose than this example, you should understand the data license, described at http://cocodataset.org/#termsofuse"

In [ ]:
import os
import urllib.request

def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)


# MSCOCO validation image files
download('http://images.cocodataset.org/zips/val2017.zip')

In [ ]:
!mkdir data
!mkdir data/valid
!unzip -qo val2017.zip -d data
!mv data/val2017 data/train
!mv data/train/0000002* data/valid
!mv data/train/0000005* data/valid

### 使用するPyTorchソースコードのダウンロード

In [ ]:
!git clone https://github.com/pytorch/examples.git
! mv examples/super_resolution ./
!rm -rf examples

### 画像をS3にアップロード

In [ ]:
s3_data = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print(s3_data)

### 画像のS3パスを設定
学習時にこのパスを使用して学習用画像を指定します。

In [ ]:
# s3_data = 's3://' + bucket + '/' + prefix
s3_train_data = s3_data + '/train'
s3_valid_data = s3_data + '/valid'

## ソースコードの書き換え
先ほどダウンロードしたソースコードは，Amazon SageMakerで使用できる状態にはなっていません。

SageMakerを使って学習，推論させるためにソースコードを書き換える必要があります。
 
Jupyterのファイルブラウザを表示し，`super_resolution/main.py`と`super_resolution/data.py`の2つのファイルを開きます。



### 不要箇所の削除（main.py）
別の部分に記述を移動するため不要となる12から43行目を全て削除します。

```python
parser = argparse.ArgumentParser(description='PyTorch Super Res Example')
parser.add_argument('--upscale_factor', type=int, required=True, help="super resolution upscale factor")
parser.add_argument('--batchSize', type=int, default=64, help='training batch size')
parser.add_argument('--testBatchSize', type=int, default=10, help='testing batch size')
（中略）
print('===> Building model')
model = Net(upscale_factor=opt.upscale_factor).to(device)
criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=opt.lr)
```

次に，main.pyの一番下のこの部分のコードを削除します。

```python
for epoch in range(1, opt.nEpochs + 1):
    train(epoch)
    test()
    checkpoint(epoch)
```

### 各種ライブラリとログ取得用の記述を追加（main.py）
SageMaker，PyTorch，Pythonライブラリを使用するための記述と，ログをCloudWatchに出力するための記述を追加します。

先ほど削除したコードがあったあたりに以下の記述を追加します。
```python
import json
import logging
import sys
import os
from PIL import Image
import numpy as np
from torchvision.transforms import ToTensor

from sagemaker_containers.beta.framework import (content_types, encoders, env, modules, transformer,
                                                 worker)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))
```

### train()関数の変更（main.py）
取得した環境変数の値を使って学習するようtrain()関数を変更します。
```python
def train(opt):
    
    torch.manual_seed(opt.seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print('===> Loading datasets')
    train_set = get_training_set(opt.upscale_factor, opt.train_dir)
    test_set = get_test_set(opt.upscale_factor, opt.valid_dir)
    training_data_loader = DataLoader(dataset=train_set, num_workers=4, batch_size=opt.batch_size, shuffle=True)
    testing_data_loader = DataLoader(dataset=test_set, num_workers=4, batch_size=opt.test_batch_size, shuffle=False)

    print('===> Building model')
    model = Net(upscale_factor=opt.upscale_factor).to(device)
    criterion = nn.MSELoss()

    optimizer = optim.Adam(model.parameters(), lr=opt.lr)

    num_epoch = opt.epochs
    for epoch in range(1, num_epoch + 1):

        epoch_loss = 0
        for iteration, batch in enumerate(training_data_loader, 1):
            input, target = batch[0].to(device), batch[1].to(device)

            optimizer.zero_grad()
            loss = criterion(model(input), target)
            epoch_loss += loss.item()
            loss.backward()
            optimizer.step()

            print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(training_data_loader), loss.item()))

        print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(training_data_loader)))

        test(testing_data_loader, model, criterion)
        checkpoint(epoch, model, opt.model_dir)
    save_model(model, opt.model_dir)
```

### 推論時に使用する関数とモデルを定義する関数の追加（main.py）
モデルを定義する関数`predict_fn()`は必ず実装が必要です。モデルのネットワーク構成を定義し，重みデータをロードしてモデルを返します。

`input_fn(), predict_fn(), output_fn()`を実装しない場合は，[デフォルトの処理](https://github.com/aws/sagemaker-pytorch-serving-container/blob/master/src/sagemaker_pytorch_serving_container/default_inference_handler.py)が実行されます。`input_fn()`で推論データを受け取って，データをモデルに入力できる形に変換します。`predict_fn()`は`input_fn()`の出力を受け取ってそれを入力として推論を行い，その結果を返します。`output_fn()`は`predict_fn()`の出力を受け取り，推論リクエストへのレスポンスとして返します。

今回は，受け取った入力データ（画像）を`input_fn()`でnumpyにデコードしたのちPILImageに変換し，`predict_fn()`で変換したPILImageを使って推論してその結果をnumpyに変換し，`output_fn()`で推論結果を返します。

main.pyの一番下に以下のコードを追加します。

```python
def input_fn(input_data, content_type):
    logger.info('input_fn---')
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    np_array = encoders.decode(input_data, content_type)
    
    image = Image.fromarray(np.uint8(np_array)).convert('YCbCr')

    return image

def predict_fn(data, model):
    logger.info("input type: " + str(data.mode))
    y, cb, cr = data.split()
    
    with torch.no_grad():
        image = ToTensor()(y).view(1, -1, y.size[1], y.size[0])
        
    if torch.cuda.is_available():
        image = image.cuda()
        
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    input_data = image.to(device)
    model.eval()
    with torch.no_grad():
        out = model(input_data)
        
    out = out.cpu()
    out_img_y = out[0].detach().numpy()
    out_img_y *= 255.0
    out_img_y = out_img_y.clip(0, 255)
    out_img_y = Image.fromarray(np.uint8(out_img_y[0]), mode='L')

    out_img_cb = cb.resize(out_img_y.size, Image.BICUBIC)
    out_img_cr = cr.resize(out_img_y.size, Image.BICUBIC)
    out_img = Image.merge('YCbCr', [out_img_y, out_img_cb, out_img_cr]).convert('RGB')

    output = np.asarray(out_img)

    return output

def output_fn(prediction, accept):
    logger.info('output_fn--')
    logger.info(accept)
    
    logger.info('predict size: ' + str(np.shape(prediction)))

    return worker.Response(response=encoders.encode(prediction, accept), mimetype=accept)

def model_fn(model_dir):
    logger.info('model_fn---')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    UPSCALE_FACTOR = 2
    model = Net(upscale_factor=UPSCALE_FACTOR).to(device)
    
    if torch.cuda.is_available():
        model.cuda()
        with open(os.path.join(model_dir, 'model.pth'), 'rb') as f:
            model.load_state_dict(torch.load(f))
    else:
        with open(os.path.join(model_dir, 'model.pth'), 'rb') as f:
            model.load_state_dict(torch.load(f, map_location=lambda storage, loc: storage))

    return model
```

### test()関数を変更（main.py）
train()関数から呼び出されるtest()関数を，パラメタを引数で与えて以下のように変更します。
```python
def test(testing_data_loader, model, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    avg_psnr = 0
    with torch.no_grad():
        for batch in testing_data_loader:
            input, target = batch[0].to(device), batch[1].to(device)

            prediction = model(input)
            mse = criterion(prediction, target)
            psnr = 10 * log10(1 / mse.item())
            avg_psnr += psnr
    psnr_res = avg_psnr / len(testing_data_loader)
    
    print("===> Avg. PSNR: {:.4f} dB".format(psnr_res))
```

### モデルを保存する関数を追加（main.py）
学習したモデルを保存するための関数を追加します。
main.pyの一番下に以下のコードを追加します。
```python
def save_model(model, model_dir):
    print("Saving the model.")
    path = os.path.join(model_dir, 'model.pth')
    # recommended way from http://pytorch.org/docs/master/notes/serialization.html
    torch.save(model.cpu().state_dict(), path)
```

### checkpointを保存する関数を変更（main.py）
train()関数から呼び出されるcheckpoint()関数を，パラメタを引数で与えて以下のように変更します。
```python
def checkpoint(epoch, model, model_dir):
    model_out_path = "{}/model_epoch_{}.pth".format(model_dir, epoch)
    torch.save(model, model_out_path)
    print("Checkpoint saved to {}".format(model_out_path))
```


### 環境変数の取得（main.py）
学習時に学習用コンテナに渡される環境変数とハイパーパラメタを取得します。
main.pyの一番下に以下のコードを追加します。
```python
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Train Super Resolution Models')
    # Data and model checkpoints directories
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=64, metavar='N',
                        help='input batch size for testing (default: 64)')
    parser.add_argument('--epochs', type=int, default=10, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                        help='learning rate (default: 0.01)')
    parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--backend', type=str, default=None,
                        help='backend for distributed training (tcp, gloo on cpu and gloo, nccl on gpu)')
    parser.add_argument('--upscale_factor', type=int, default=2,
                        help='upscale factor (default: 2)')

    # Container environment
    parser.add_argument('--hosts', type=list, default=json.loads(os.environ['SM_HOSTS']))
    parser.add_argument('--current-host', type=str, default=os.environ['SM_CURRENT_HOST'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train-dir', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--valid-dir', type=str, default=os.environ['SM_CHANNEL_VALID'])
    parser.add_argument('--num-gpus', type=int, default=os.environ['SM_NUM_GPUS'])
    
    train(parser.parse_args())
```

### main.pyから呼ばれる関数の変更（data.py）
引数を使って関数を呼び出すよう，get_training_set()関数と，get_test_set()関数を書き換えます。

```python
def get_training_set(upscale_factor, train_dir):
    crop_size = calculate_valid_crop_size(256, upscale_factor)

    return DatasetFromFolder(train_dir,
                             input_transform=input_transform(crop_size, upscale_factor),
                             target_transform=target_transform(crop_size))


def get_test_set(upscale_factor, test_dir):
    crop_size = calculate_valid_crop_size(256, upscale_factor)

    return DatasetFromFolder(test_dir,
                             input_transform=input_transform(crop_size, upscale_factor),
                             target_transform=target_transform(crop_size))
```


## 学習
画像の準備が終わったら、学習を行います。

### PyTorch の estimator を定義
PyTorch のコードや、学習に使用するインスタンスタイプ、インスタンス数、ハイパーパラメタなどを設定します。

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='main.py',
                    source_dir='./super_resolution',
                    role=role,
                    framework_version='1.1.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters={
                        'epochs': 2,
                        'backend': 'gloo',
                        'upscale_factor': 2
                    },
                   metric_definitions=[
                   {'Name': 'training:Loss', 'Regex': '===> Epoch .*? Complete: Avg. Loss: (.*?)$'},
                    {'Name': 'validation:PSNR', 'Regex': '===> Avg. PSNR: (.*?) dB'}
                ])

### 学習開始
estimator に対して fit 関数を呼ぶことで学習が開始します。

学習用インスタンスが立ち上がり、 PyTorch のコンテナとソースコードがインスタンスにロードされ、学習が開始します。

In [ ]:
estimator.fit({'train': s3_train_data, 'valid': s3_valid_data})

## リアルタイム推論
学習が完了したら，学習したモデルを使って推論を行います。

ここでは，リアルタイム推論の手順を説明します。
### 推論用エンドポイント起動
estimator に対して deploy 関数を実行すると、推論用エンドポイントが立ち上がります。

エンドポイントが立ち上がるまでに１０分程度かかることがあります。

deploy を実行したのち、進捗を示すハイフンが表示されたのちビックリマークが表示されたらエンドポイントが立ち上がったことを示します。

推論用エンドポイントが立ち上がると、train.py の model_fn でモデルが構成され、学習した重みがロードされます。


In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')

### 推論の実行
エンドポイントに画像を送って、結果を取得します。今回使用するモデルは，入力画像を縦横2倍にした超解像画像を出力します。

画像がエンドポイントに送られると train.py の input_fn が実行され、その結果が predict_fn（今回は定義なしなのでデフォルトの動作が実行される）に入力されて推論が実行され、推論結果が output_fn に渡されて numpy に変換されて返ってきます。

試しに、検証に使用した画像を１枚エンドポイントに送ってその結果を画像として保存します。このノートブックと同じフォルダに画像が保存されます。

ハンズオンではほとんど学習が進んでいないので結果の画像はいまひとつだと思いますが、学習を進めれば良い結果が得られます。

In [ ]:

from PIL import Image
import numpy as np
import os

filename = 'data/valid/000000201676.jpg'
filename_base = os.path.basename(filename)
image = Image.open(filename)
image = np.asarray(image)

srimage = predictor.predict(image)
pil_img = Image.fromarray(srimage)

outfilename = 'out_srf_' + str(2) + '_' + filename_base
pil_img.save(outfilename)
print(outfilename)

## エンドポイントの削除
エンドポイントは起動している間課金され続けるので、不要になったら削除します。

コンソールからも削除することが可能です。

In [ ]:
predictor.delete_endpoint()

## [参考] 過去に学習したモデルから推論用エンドポイントを立てる
過去に学習したモデルを使用したい場合があります。その場合は、PyTorchModel を使用してモデルを定義して deploy 関数を呼びます。

以下のセルの`<your bucket> <your folder>`の部分を実際のものに書き換えてから実行してください。

In [ ]:
from sagemaker.pytorch.model import PyTorchModel
pytorch_model = PyTorchModel(
    model_data='s3://<your bucket>/<your folder>/model.tar.gz', 
    role=role,
    framework_version='1.1.0',
    entry_point='train.py')

In [ ]:
predictor = pytorch_model.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')

In [ ]:
predictor.delete_endpoint()

## 不要なリソースの削除
本ハンズオンを実施すると，ノートブックインスタンス，推論用エンドポイント，S3において課金が発生した状態となります。

不要なリソースは削除してください。削除方法は[こちら](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/ex1-cleanup.html)をご参照ください。

## 参考情報

### 本サンプルで使用した SRGAN コード
こちらのPyTorch Exampleのソースコードを使用させていただきました。
- https://github.com/pytorch/examples/tree/master/super_resolution

### Amazon SageMaker の使い方を学ぶのに便利なサンプルノートブック
- 画像分類　https://github.com/ohbuchim/demo-notebooks/blob/master/image_classification/Image-classification-transfer.ipynb
 - 画像分類のビルトインアルゴリズムを使用して，学習，ハイパーパラメータチューニング，リアルタイム推論，バッチ推論を行うノートブックです
- 公式サンプルノートブック GitHub　https://github.com/awslabs/amazon-sagemaker-examples

### Amazon SageMaker PyTorch コンテナ
- コンテナ　　https://github.com/aws/sagemaker-pytorch-container
- Servingコンテナ　　https://github.com/aws/sagemaker-pytorch-serving-container/blob/master/src/sagemaker_pytorch_serving_container/

### 分散学習に関する情報
- https://aws.amazon.com/jp/blogs/news/amazon-sagemaker-now-supports-pytorch-and-tensorflow-1-8/
